In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv('C:/Users/bcspr/OneDrive/Desktop/Projects folder/NBA_Advanced_Stats_2002-2022.csv')
df = data.copy()
pd.set_option('display.max_columns', None)
df[['year_drop', 'name']] = df['year-name'].str.split('-', 1, expand=True)
df.drop(columns= ["Unnamed: 0", 'year_drop','year-name'], inplace= True)
df['Pos'] = df['Pos'].str.split('-').str[0]
new_order = ['name'] + [col for col in df if col != 'name']
df.dropna(inplace= True)
df = df[new_order]
df.head()


C:\Users\bcspr\AppData\Local\Temp\ipykernel_25368\2802590881.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['year_drop', 'name']] = df['year-name'].str.split('-', 1, expand=True)


,name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year
0,Tariq Abdul-Wahad,SG,28,DAL,14,204,12.4,0.470,0.017,0.103,7.6,14.0,10.8,15.4,1.5,1.1,10.3,15.0,0.2,0.2,0.4,0.104,-1.6,0.2,-1.4,0.0,2003
1,Shareef Abdur-Rahim,PF,26,ATL,81,3087,19.9,0.566,0.051,0.457,6.7,17.8,12.5,14.7,1.5,0.9,13.0,24.2,7.4,2.3,9.7,0.151,2.3,-0.7,1.6,2.8,2003
2,Courtney Alexander,PG,25,NOH,66,1360,9.3,0.459,0.113,0.289,3.3,6.8,5.0,9.8,1.2,0.3,10.7,21.3,0.1,1.0,1.1,0.040,-3.3,-1.2,-4.5,-0.9,2003
3,Malik Allen,PF,24,MIA,80,2318,9.9,0.455,0.005,0.153,6.5,14.9,10.6,4.4,0.9,2.6,13.2,19.7,-1.7,2.6,0.9,0.018,-3.9,-0.4,-4.4,-1.4,2003
4,Ray Allen,SG,27,TOT,76,2880,21.3,0.565,0.391,0.253,3.8,11.8,7.8,21.6,1.9,0.4,11.6,27.8,7.6,1.5,9.1,0.152,4.7,-1.0,3.6,4.1,2003


In [4]:
#add MVP column
year_player_dict = {
    2022: 'Nikola Jokić',
    2021: 'Nikola Jokić',
    2020: 'Giannis Antetokounmpo',
    2019: 'Giannis Antetokounmpo',
    2018: 'James Harden',
    2017: 'Russell Westbrook',
    2016: 'Stephen Curry',
    2015: 'Stephen Curry',
    2014: 'Kevin Durant',
    2013: 'LeBron James',
    2012: 'LeBron James',
    2011: 'Derrick Rose',
    2010: 'LeBron James',
    2009: 'LeBron James',
    2008: 'Kobe Bryant',
    2007: 'Dirk Nowitzki',
    2006: 'Steve Nash',
    2005: 'Steve Nash',
    2004: 'Kevin Garnett',
    2003: 'Tim Duncan'
}

# Create a new column and set values based on the dictionary
df['IsMVP'] = df.apply(lambda row: 1 if (row['year'], row['name']) in year_player_dict.items() else 0, axis=1)


In [5]:
df.head()

,name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year,IsMVP
0,Tariq Abdul-Wahad,SG,28,DAL,14,204,12.4,0.470,0.017,0.103,7.6,14.0,10.8,15.4,1.5,1.1,10.3,15.0,0.2,0.2,0.4,0.104,-1.6,0.2,-1.4,0.0,2003,0
1,Shareef Abdur-Rahim,PF,26,ATL,81,3087,19.9,0.566,0.051,0.457,6.7,17.8,12.5,14.7,1.5,0.9,13.0,24.2,7.4,2.3,9.7,0.151,2.3,-0.7,1.6,2.8,2003,0
2,Courtney Alexander,PG,25,NOH,66,1360,9.3,0.459,0.113,0.289,3.3,6.8,5.0,9.8,1.2,0.3,10.7,21.3,0.1,1.0,1.1,0.040,-3.3,-1.2,-4.5,-0.9,2003,0
3,Malik Allen,PF,24,MIA,80,2318,9.9,0.455,0.005,0.153,6.5,14.9,10.6,4.4,0.9,2.6,13.2,19.7,-1.7,2.6,0.9,0.018,-3.9,-0.4,-4.4,-1.4,2003,0
4,Ray Allen,SG,27,TOT,76,2880,21.3,0.565,0.391,0.253,3.8,11.8,7.8,21.6,1.9,0.4,11.6,27.8,7.6,1.5,9.1,0.152,4.7,-1.0,3.6,4.1,2003,0


In [6]:
df.query('IsMVP == 1')

,name,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year,IsMVP
123,Tim Duncan,PF,26,SAS,81,3181,26.9,0.564,0.016,0.455,9.9,27.3,19.0,19.5,0.9,5.2,12.9,28.0,9.5,6.9,16.5,0.248,5.0,2.5,7.6,7.7,2003,1
667,Kevin Garnett,PF,27,MIN,82,3231,29.4,0.547,0.027,0.289,9.1,30.0,20.1,24.4,2.0,4.0,10.5,29.6,10.4,8.0,18.3,0.272,6.8,3.3,10.2,10.0,2004,1
1444,Steve Nash,PG,30,PHO,75,2573,22.0,0.606,0.254,0.278,2.5,7.6,5.2,49.2,1.4,0.2,20.3,20.5,9.7,1.2,10.9,0.203,5.5,-0.8,4.7,4.4,2005,1
2011,Steve Nash,PG,31,PHO,79,2796,23.3,0.632,0.324,0.264,1.9,11.1,6.6,44.4,1.1,0.3,19.0,23.3,10.3,2.1,12.4,0.212,5.7,-0.7,5.0,4.9,2006,1
2559,Dirk Nowitzki,PF,28,DAL,78,2820,27.6,0.605,0.129,0.411,5.3,23.9,14.8,17.8,1.0,1.7,9.5,28.9,11.8,4.5,16.3,0.278,7.2,1.2,8.3,7.4,2007,1
2814,Kobe Bryant,SG,29,LAL,82,3192,24.2,0.576,0.246,0.439,3.4,14.1,9.0,23.9,2.4,0.9,11.3,31.4,9.5,4.3,13.8,0.208,5.2,0.6,5.8,6.3,2008,1
3599,LeBron James,SF,24,CLE,81,3054,31.7,0.591,0.238,0.472,4.3,19.0,11.9,38.0,2.4,2.4,11.0,33.8,13.7,6.5,20.3,0.318,9.5,3.7,13.2,11.8,2009,1
4194,LeBron James,SF,25,CLE,76,2966,31.1,0.604,0.253,0.506,3.0,18.5,11.1,41.8,2.2,2.0,12.3,33.5,13.3,5.2,18.5,0.299,9.1,2.8,11.8,10.3,2010,1
4977,Derrick Rose,PG,22,CHI,81,3026,23.5,0.550,0.241,0.348,3.2,9.4,6.4,38.7,1.5,1.3,13.1,32.2,8.3,4.8,13.1,0.208,6.3,0.5,6.8,6.7,2011,1
5381,LeBron James,SF,27,MIA,62,2326,30.7,0.605,0.127,0.429,5.0,19.7,12.6,33.6,2.6,1.7,13.3,32.0,10.0,4.5,14.5,0.298,8.2,2.7,10.9,7.6,2012,1


In [7]:
model_construction = df.drop(columns= ['name','Tm'])
model_construction = pd.get_dummies(model_construction, 'Pos')
model_construction

,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year,IsMVP,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG
0,28,14,204,12.4,0.470,0.017,0.103,7.6,14.0,10.8,15.4,1.5,1.1,10.3,15.0,0.2,0.2,0.4,0.104,-1.6,0.2,-1.4,0.0,2003,0,0,0,0,0,1
1,26,81,3087,19.9,0.566,0.051,0.457,6.7,17.8,12.5,14.7,1.5,0.9,13.0,24.2,7.4,2.3,9.7,0.151,2.3,-0.7,1.6,2.8,2003,0,0,1,0,0,0
2,25,66,1360,9.3,0.459,0.113,0.289,3.3,6.8,5.0,9.8,1.2,0.3,10.7,21.3,0.1,1.0,1.1,0.040,-3.3,-1.2,-4.5,-0.9,2003,0,0,0,1,0,0
3,24,80,2318,9.9,0.455,0.005,0.153,6.5,14.9,10.6,4.4,0.9,2.6,13.2,19.7,-1.7,2.6,0.9,0.018,-3.9,-0.4,-4.4,-1.4,2003,0,0,1,0,0,0
4,27,76,2880,21.3,0.565,0.391,0.253,3.8,11.8,7.8,21.6,1.9,0.4,11.6,27.8,7.6,1.5,9.1,0.152,4.7,-1.0,3.6,4.1,2003,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12206,33,26,475,15.8,0.526,0.299,0.188,8.7,18.1,13.1,13.6,3.3,2.3,12.4,16.0,0.5,0.8,1.3,0.127,-0.2,2.2,2.0,0.5,2022,0,0,1,0,0,0
12207,23,76,2652,25.4,0.603,0.395,0.358,2.1,9.6,5.9,46.7,1.3,0.2,14.5,34.4,9.0,1.0,10.0,0.181,7.1,-2.0,5.2,4.8,2022,0,0,0,1,0,0
12208,23,56,706,17.4,0.546,0.045,0.247,13.9,33.0,23.6,11.0,1.2,3.0,13.0,19.9,0.8,1.4,2.1,0.145,-1.4,0.4,-1.0,0.2,2022,0,1,0,0,0,0
12209,29,27,355,17.2,0.627,0.044,0.544,14.9,24.1,19.3,9.4,1.1,1.7,14.6,15.9,0.9,0.2,1.1,0.143,-1.2,-1.0,-2.2,0.0,2022,0,1,0,0,0,0


In [8]:
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data
    ('clf', LogisticRegression()) #step2 - classifier
])
pipeline.steps

X = model_construction.copy()
X = X.drop(columns= "IsMVP")
y = model_construction['IsMVP']




In [9]:
null_rows = df[df.isnull().any(axis=1)]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y
                                                   ,
                                                   test_size = 0.4,
                                                   random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7283, 29)
(4856, 29)
(7283,)
(4856,)


In [11]:
scores = cross_validate(pipeline,X_train,y_train)
print(scores)
mean_score = scores['test_score'].mean()
print('mean score: ' + str(mean_score))

{'fit_time': array([0.038764  , 0.02876544, 0.02591991, 0.02407503, 0.02783489]), 'score_time': array([0.00270677, 0.00207472, 0.0020175 , 0.00204468, 0.00208902]), 'test_score': array([0.99931366, 0.99862732, 0.99794097, 0.99587912, 0.99931319])}
mean score: 0.998214851380603


In [21]:
clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    if hasattr(classifier, 'feature_importances_'):
        feature_importances = classifier.feature_importances_
        model_name = str(classifier).split('(')[0]  # Extract model name
        
        file_name = f"{model_name}_featureimportance.txt"
        
        np.savetxt(file_name, feature_importances, delimiter=',')
        
        print(f'Feature importances saved as {file_name}')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())

---------------------------------
LogisticRegression()
-----------------------------------
fit_time  mean  0.025796270370483397
fit_time  std  0.00168140129522759
score_time  mean  0.001914644241333008
score_time  std  0.0007456388742581997
test_score  mean  0.998214851380603
test_score  std  0.0012739301402737403
---------------------------------
SVC()
-----------------------------------
fit_time  mean  0.0441678524017334
fit_time  std  0.0021220271621883403
score_time  mean  0.01970372200012207
score_time  std  0.0010030942533411505
test_score  mean  0.9984896709330477
test_score  std  0.00027434848782535313
---------------------------------
KNeighborsClassifier(n_neighbors=3)
-----------------------------------
fit_time  mean  0.00803375244140625
fit_time  std  0.0016013296426040078
score_time  mean  0.08137812614440917
score_time  std  0.0034131201805004253
test_score  mean  0.9984896709330477
test_score  std  0.00027434848782535313
---------------------------------
DecisionTreeCla

In [13]:
pipeline.set_params(clf= SVC())
pipeline.steps

[('normalizer', StandardScaler()), ('clf', SVC())]

In [14]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    'clf__kernel' : ['linear', 'rbf'],
    'clf__C' : np.linspace(0.1,1.2,12)
})
cv_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('normalizer', StandardScaler()),
                                       ('clf', SVC())]),
             param_grid={'clf__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2]),
                         'clf__kernel': ['linear', 'rbf']})

In [ ]:
cv_grid.best_estimator_

Pipeline(steps=[('normalizer', StandardScaler()), ('clf', SVC(C=0.1))])

In [ ]:
cv_grid.best_score_

0.9984896709330477

In [ ]:
y_predict = cv_grid.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)
accuracy


0.9981466227347611